### Convolution

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader

sys.path.append('..')
import utils

### Get data

In [ ]:
x_train, y_train, x_valid, y_valid = utils.get_mnist('../data')

bs = 3

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs)

### Get a batch from the training set

In [ ]:
train_iter = iter(train_dl)  # an iterator
xb, yb = next(train_iter)

In [ ]:
# a batch of 3 samples (each sample has size 28*28)
xb.shape

### Change the size of the samples in the batch
 + there are 3 samples in the batch
 + there is 1 input channel
 + each image has size 28*28

In [ ]:
xb = xb.view(-1, 1, 28, 28)
xb.shape

### Define a convolution object
 + `in_channels`: number of image channels (3 for RGB, 1 for Grayscale)
 + `out_channels`: number of different masks to apply to each input channel
 + `kernel_size`: size of the square kernel
 + `padding`: number of zeros added on all sides of the image

In [ ]:
ks = 3
conv = torch.nn.Conv2d(1, 5, kernel_size=ks, stride=1, padding=1)

### Examine the weights and bias for the k-th mask (initialized with random values).

In [ ]:
k = 0
conv.weight[k].data, conv.bias[k].data

### Apply a convolution to the samples of the batch.
 + there are 3 samples in the batch
 + there are 5 output channels
 + each image has size 28*28

In [ ]:
c = conv(xb)
c.shape

### Show the output channels for all samples in the batch.

In [ ]:
# plot the original images in the batch
utils.show_random_samples(xb, rows=1, cols=bs, shuffle=False)

# plot the result of the convolution
for k in range(bs):
    utils.show_random_samples(c[k].detach(), rows=1, cols=conv.out_channels, shuffle=False)

### Example from http://cs231n.github.io/convolutional-networks/
 + When there are three input layers and two output layers, there are:
  + 3 weighting matrices for each output (hence 6 in total)
  + 1 bias vector per output (hence 2 in total)

In [ ]:
def apply_convolution(x0, x1, x2, w0, w1, w2, b):
    """Apply convolution (explicit) example.
    
    x0, x1, x2: 7x7 images (three channels)
    w0, w1, w2: 3x3 kernels
    b: bias
    """
    n = 3  # kernel size
    stride = 2
    out = np.ndarray((3,3), dtype=np.int)

    k1 = 0
    for i in range(0, 6, stride):
        k2 = 0
        for j in range(0, 6, stride):
            out[k1,k2] = np.sum(x0[i:i+n, j:j+n] * w0 + 
                                x1[i:i+n, j:j+n] * w1 + 
                                x2[i:i+n, j:j+n] * w2) + b
            out[k1,k2] = np.sum(x0[i:i+n, j:j+n] * w0 + 
                                x1[i:i+n, j:j+n] * w1 + 
                                x2[i:i+n, j:j+n] * w2) + b
            out[k1,k2] = np.sum(x0[i:i+n, j:j+n] * w0 + 
                                x1[i:i+n, j:j+n] * w1 + 
                                x2[i:i+n, j:j+n] * w2) + b
            k2 += 1
        k1 += 1

    return out

In [ ]:
x0 = np.array([[0,0,0,0,0,0,0], [0,1,0,1,2,2,0], [0,2,2,2,0,2,0], 
               [0,0,0,0,0,1,0], [0,1,2,0,1,0,0], [0,1,1,2,1,1,0], [0,0,0,0,0,0,0]])

x1 = np.array([[0,0,0,0,0,0,0], [0,1,0,2,2,0,0], [0,1,1,2,2,0,0], 
               [0,0,0,2,2,2,0], [0,2,1,2,0,2,0], [0,1,0,0,2,2,0], [0,0,0,0,0,0,0]])

x2 = np.array([[0,0,0,0,0,0,0], [0,0,0,2,0,0,0], [0,1,2,2,0,1,0], 
               [0,1,1,0,1,0,0], [0,2,0,1,2,2,0],  [0,2,0,0,1,0,0], [0,0,0,0,0,0,0]])

b0 = 1
w00 = np.array([[0,0,-1], [1,1,1], [0,1,0]])
w01 = np.array([[1,1,0], [0,0,0], [1,1,0]])
w02 = np.array([[-1,1,-1], [0,0,1], [1,-1,1]])

b1 = 0
w10 = np.array([[1,1,0], [1,-1,0], [-1,-1,-1]])
w11 = np.array([[-1,-1,1], [0,-1,1], [1,1,1]])
w12 = np.array([[-1,-1,1], [0,0,1], [0,-1,-1]])

X = torch.FloatTensor([x0,x1,x2]).view(-1,3,7,7)
conv = torch.nn.Conv2d(3, 2, kernel_size=3, stride=2, padding=0)

conv.weight[0] = torch.tensor([w00, w01, w02])
conv.weight[1] = torch.tensor([w10, w11, w12])
conv.bias[0] = torch.tensor([b0])
conv.bias[1] = torch.tensor([b1])

print(X.shape)

# perform convolution manually
print(apply_convolution(x0, x1, x2, w00, w01, w02, b0))
print(apply_convolution(x0, x1, x2, w10, w11, w12, b1))

# perform convolution using pytorch
conv(X).detach()